In [1]:
!pip install bitsandbytes

import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
augment_dataset = False

In [ ]:
df1 = pd.read_csv("archaic_to_english_dataset.csv")
df1 = df1.rename(columns={"archaic": "source", "modern": "target"}) if "archaic" in df1.columns else df1

if augment_dataset:
  df2 = pd.read_csv("archaic_to_english_dataset_added.csv")
  df2 = df2.rename(columns={"archaic_italian": "source", "english_translation": "target"})
  df = pd.concat([df1, df2], ignore_index=True)
  df = df.drop_duplicates()
else:
  df = df1


dataset = DatasetDict({
    "train": Dataset.from_pandas(df.sample(frac=0.9, random_state=42).reset_index(drop=True)),
    "test": Dataset.from_pandas(df.sample(frac=0.1, random_state=42).reset_index(drop=True))
})

In [3]:
model_name = "Helsinki-NLP/opus-mt-it-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(80379, 512, padding_idx=80378)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(80379, 512, padding_idx=80378)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [4]:
def preprocess(example):
    inputs = tokenizer(example["source"], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(example["target"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [5]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_finetuned",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-5-89f00b79d6d9>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [6]:
trainer.train()

model.save_pretrained("./marian_finetuned_a")
tokenizer.save_pretrained("./marian_finetuned_a")

Step,Training Loss
10,8.856700


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[80378]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./marian_finetuned_a/tokenizer_config.json',
 './marian_finetuned_a/special_tokens_map.json',
 './marian_finetuned_a/vocab.json',
 './marian_finetuned_a/source.spm',
 './marian_finetuned_a/target.spm',
 './marian_finetuned_a/added_tokens.json')

INFERENCE

In [7]:
model_it_en = MarianMTModel.from_pretrained("./marian_finetuned_a").to(device)
tokenizer_it_en = MarianTokenizer.from_pretrained("./marian_finetuned_a")

model_en_it = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-it").to(device)
tokenizer_en_it = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
!pip install bitsandbytes

In [13]:
from prometheus import PrometheusEval_AtM

evaluator = PrometheusEval_AtM(quantized = True, device = device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
df = pd.read_csv("test.csv", sep=";")
df = df.rename(columns={"Sentence": "source", "Traductions": "target"})

dataset = Dataset.from_pandas(df)

In [17]:
import re
i = 1

source_sentences = []
predicted_sentences = []
gold_sentences = [] #annoted by hand
prometheus_score = []
df_col = ["source_sentences", "predicted_sentences", "gold_sentences", "prometheus_score","GPT_score","user_score"]

print(len(dataset))
for sample in dataset:


    input_sentence = sample["source"]
    target_sentence = sample["target"]

    author = sample["Author"]
    date = sample["Date"]
    region = sample["Region"]

    # archaic it -> en
    inputs = tokenizer_it_en([input_sentence], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_en_ids = model_it_en.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    translated_en = tokenizer_it_en.decode(translated_en_ids[0], skip_special_tokens=True)

    # en -> modern it
    inputs_en_it = tokenizer_en_it([translated_en], return_tensors="pt", padding=True, truncation=True).to(device)
    translated_it_ids = model_en_it.generate(**inputs_en_it, max_length=128, num_beams=4, early_stopping=True)
    translation = tokenizer_en_it.decode(translated_it_ids[0], skip_special_tokens=True)

    source_sentences.append(input_sentence)
    predicted_sentences.append(translation)
    gold_sentences.append(target_sentence)

    evaluation = evaluator.getEvaluation(input_sentence, translation, target_sentence)
    match_ = re.search(r'\[RESULT\]\s*(\d)', evaluation)
    if match_:
      result = int(match_.group(1))

    else:
      result = 0

    prometheus_score.append(result)



    print(f"Sentence {i}")
    print(f"\tItaliano Arcaico            -> {input_sentence}")
    print(f"\tItaliano moderno            -> {translation}")
    print(f"\tGOLD LABEL                  -> {target_sentence}")
    print(f"\tPROMETHEUS EVALUATION       -> {result}")
    print(f"-----------------------------------------")
    i+=1

z = [0 for _ in range(len(dataset))]
GPT_score,user_score = z,z
df = pd.DataFrame(list(zip(source_sentences, predicted_sentences, gold_sentences,prometheus_score,GPT_score,user_score)), columns=df_col)

df.to_csv("test_results.csv", sep=";")

10


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 1
	Italiano Arcaico -> Et se l' occhio è nobile membro del corpo dell' uomo, dunque la salutazione è nobile parte della pistola, c' altressì allumina tutta la lettera come l' occhio allumina l' uomo.
	Italiano moderno -> E se l'occhio è un nobile membro del corpo dell'uomo, allora il saluto è una parte nobile della pistola, c'è anche alluminio tutta la lettera come l'uomo occhio di alluminio.
	GOLD LABEL       -> E se l’occhio è una parte nobile del corpo umano, allora il saluto è una parte nobile della lettera, che illumina l’intero testo come l’occhio illumina l’uomo.
	EVALUATION       -> 1
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 2
	Italiano Arcaico -> Tarentini, i quali erano nati di quegli di Lacedemonia et facta da lloro nobile cittade de' Greci.
	Italiano moderno -> Tarentini, che sono nati da quelli di Lacedemonia et facta dalla loro nobile cittadella dei Greci.
	GOLD LABEL       -> I Tarantini, nati dai lacedemoni, avevano fondato la loro nobile città greca.
	EVALUATION       -> 3
-----------------------------------------


...

Sentence 8
	Italiano Arcaico -> pregollo che lo liberasse di quella obbligazione, in che egli l' aveva lasciato ubbligato. El gentile uomo assentì, e liberollo, e fecene carta.
	Italiano moderno -> Lo supplico di liberarlo da quel legame, nel quale lo aveva lasciato legato; il gentiluomo si astentò, lo liberò e lo fece carta.
	GOLD LABEL       -> Gli chiese di essere liberato da quell’obbligo che gli era stato imposto. Il nobile uomo acconsentì, lo liberò, e ne redasse un documento.
	EVALUATION       -> 4
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 9
	Italiano Arcaico -> L'oro verrà dall'Aquilone. Che figuriamo noi per l'Aquilone, se non il populo Gentile congelato dal freddo del peccato, il qual populo tenne sotto il giogo della sua tirannia
	Italiano moderno -> L'oro uscirà dall'aquilone, che considereremo per l'aquilone, eccetto il popolo, Gentile, congelato dal freddo del peccato, che gli uomini custodivano sotto il giogo della sua tirannia.
	GOLD LABEL       -> L’oro verrà dal Settentrione. Cosa rappresenta il Settentrione se non il popolo pagano, congelato dal freddo del peccato, che fu tenuto sotto il giogo della sua tirannia?
	EVALUATION       -> 2
-----------------------------------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Sentence 10
	Italiano Arcaico -> sia in mezzo tra me e te: con noi non puo' tu già più lungamente dimorare, ch'io non lo sofferrò e non lo lascerò.
	Italiano moderno -> stai in mezzo a me e a te; con noi non puoi più dimorare, perché non l'ho rattristato e non lo lascerò.
	GOLD LABEL       -> Sia in mezzo tra me e te: con noi non puoi più rimanere a lungo, perché io non lo permetterò e non lo tollererò oltre.
	EVALUATION       -> 3
-----------------------------------------
